In [ ]:
# Instalando as libs necessárias.
%pip install fuzzywuzzy 

In [ ]:
# Importando os módulos.
import pandas as pd
import numpy as np
import pandas_profiling
import nltk
import warnings
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Dataset principal.
df = pd.read_csv("drive/My Drive/Colab Notebooks/NLP/wiki_movie_plots_deduped.csv")

In [ ]:
# Importando as colunas a serem utilizadas.
df = pd.DataFrame(df,columns=["Title","Genre","Plot"])

In [ ]:
# Vamos utilizar um outro Dataset com os genêros já padronizados.
df_2 = pd.read_csv("drive/My Drive/Colab Notebooks/NLP/movie_metadata.csv")

In [ ]:
# Queremos somente a coluna com a descrição dos genêros.
df_2 = pd.DataFrame(df_2,columns=["genres"])

Visualizando o primeiro DataFrame.

In [ ]:
df.head(20)

Visualizando o segundo DataFrame.

In [ ]:
df_2.head(20)

* A coluna "Genre" do primeiro DataFrame é onde se encontra os dados com problemas de escrita.
* Na coluna "genres" os dados já estão limpos e devidamente padronizados.

In [ ]:
df_ = df_2.drop_duplicates()

Vamos dar uma olhada na quantidade de valores únicos que as essas colunas possuem.

In [ ]:
print(df["Genre"].unique().shape[0])

In [ ]:
print(df_2["genres"].unique().shape[0])

Visualizando alguns dados.

In [ ]:
df["Genre"].unique().tolist()[:20]

In [ ]:
df_2["genres"].unique().tolist()[:20]

Nossa coluna "Genre" possui dados que estão separados por vírgula, alguns por traços, ou seja, não existe nenhum padrão. Podemos resolver isso facilmente removendo todos os caracteres que são estranhos.

In [ ]:
df["Genre"] = df["Genre"].str.replace('[^A-Za-z]+', ' ')

Temos também valores que são indefinidos, nesse caso vamos transformar esse valores em branco.

In [ ]:
df["Genre"] = df["Genre"].str.replace('unknown', ' ')

Vamos verificar como ficou nosso dataset depois dessas transformações.

In [ ]:
df["Genre"].unique().tolist()[:20]

Agora vamos criar uma função para aplicar a biblioteca fuzzywuzzy.
Esta função utiliza a biblioteca fuzzywuzzy para comparar cada genêro com
uma String que está correta, caso haja algum problema de grafia a string é substituída pela forma correta.

In [ ]:
def replace_matches_in_column(data, column, string_match, min_ratio, only_test=False):

    # lista de strings únicas.
    strings = data[column].unique()

    # Definindo 10 como o número limite de strings mais correspondentes.
    matches = process.extract(string_match, strings,
                                         limit=10,
                                         scorer=fuzz.token_sort_ratio)

    # Obtendo as correspondencias que são maiores que o min_ratio.
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    if len(close_matches) > 0:
        print(string_match, ':', close_matches)

    # Obtendo as linhas que não correspondem ao limite definido.
    if not(only_test):
        rows_with_matches = data[column].isin(close_matches)

        # Substituindo todos os valores correspondentes pelo valor de entrada.
        # Os valores que não são correspondentes irão permanecer os mesmos.
        data.loc[rows_with_matches, column] = string_match

In [ ]:
# Eu utilizei uma taxa de semelhança  de 80%.
for marca in df_2['genres']:
     replace_matches_in_column(data=df, column="Genre", 
                             string_match=marca, min_ratio=80)

Agora que terminamos de aplicar a nossa função e encontrar a similaridade entre os nossos dados, vamos verificar quantos valores únicos nosso dataset possui agora.

In [ ]:
print(df["Genre"].unique().shape[0])

In [ ]:
df.head(20)

Bem, podemos observar agora que tivemos uma redução consideravel no número de valores únicos em nosso dataset.

Na verdade muitos desses valores significavam a mesma coisa, porém estavam escritos de várias formas diferentes. Ainda possuimos alguns valores incorretos, mais a sua grande maioria foi tratada.